# Aporior关联算法
数据挖掘十大算法之Apriori详解 - CSDN博客 https://blog.csdn.net/baimafujinji/article/details/53456931

> 大多数关联规则挖掘算法通常采用的一种策略是，将关联规则挖掘任务分解为如下两个主要的子任务。

+ 频繁项集产生：其目标是发现满足最小支持度阈值的所有项集，这些项集称作频繁项集（frequent itemset）。
+ 规则的产生：其目标是从上一步发现的频繁项集中提取所有高置信度的规则，这些规则称作强规则（strong rule）。

+ Apriori定律1：如果一个集合是频繁项集，则它的所有子集都是频繁项集。

例如：假设一个集合{A,B}是频繁项集，即A、B同时出现在一条记录的次数大于等于最小支持度min_support，则它的子集{A},{B}出现次数必定大于等于min_support，即它的子集都是频繁项集。

+ Apriori定律2：如果一个集合不是频繁项集，则它的所有超集都不是频繁项集。

举例：假设集合{A}不是频繁项集，即A出现的次数小于 min_support，则它的任何超集如{A,B}出现的次数必定小于min_support，因此其超集必定也不是频繁项集。

## 分拆

In [1]:
import random

In [2]:
l = list('ABCD')

In [3]:
def item_set(l):
    item_set = []
    for i in range(len(l)):
        for j in range(i,len(l)):
            if l[i][0] != l[j][0] and l[i][0] not in l[j]:
                item = sorted([l[i][0]+l[j]])[0]
                if item not in item_set:
                    item_set.append(item)    
    return item_set

In [4]:
def aporior_rule_1(df,support,confidence,ms='--'):
    aporior_item = list(df.columns)
    result = DataFrame(columns=['support','confidence'])
  
    n = 0
    while len(aporior_item) > 0:
        n += 1
        aporior_item_temp = []
        print('进行第{}次关联，共{}个item，分别是{}'.format(n,len(aporior_item),aporior_item))
        for i in aporior_item:
            p_support = 1.0*df[list(i)].prod(axis=1).sum()/len(df)  #支持度
            #print('{}   {}'.format(ms.join(i),p))
            if p_support >= support:
                aporior_item_temp.append(i)   #达到最小支持度的就保留进行下一轮
                
                if len(i) >= 2: #只有一个字母直接pass
                    for k in i:
                        new_i = list(i).copy()
                        new_i.remove(k)
                        item = ms.join(new_i+[k])
                        p_confindenc = 1.0*df[list(i)].prod(axis=1).sum()/df[k].sum()
                else:
                    item = i
                    p_confindenc = 1.0*df[list(i)].prod(axis=1).sum()/df[i].sum()
                
    
                    
                if p_confindenc >= confidence:
                    result.loc[item,'support'] = p_support
                    result.loc[item,'confidence'] = p_confindenc
            
        aporior_item = item_set(aporior_item_temp)
        
    return result
        

In [5]:
def aporior_rule(df,support,confidence,ms='--'):
    aporior_item = list(df.columns)
    result = DataFrame(columns=['support','confidence'])
  
    n = 0
    while len(aporior_item) > 0:
        n += 1
        aporior_item_temp = []
        print('进行第{}次关联，共{}个item，分别是{}'.format(n,len(aporior_item),aporior_item))
        for i in aporior_item:
            p_support = 1.0*df[list(i)].prod(axis=1).sum()/len(df)  #支持度
            #print('{}   {}'.format(ms.join(i),p))
            if p_support >= support:
                aporior_item_temp.append(i)   #达到最小支持度的就保留进行下一轮
                
                for k in range(len(i)):
                    item = ms.join(list(i[0:k]+i[k+1:len(i)+1]+i[k]))
                    p_confindenc = 1.0*df[list(i)].prod(axis=1).sum()/df[i[k]].sum()  #置信度
    
                    
                if p_confindenc >= confidence:
                    result.loc[item,'support'] = p_support
                    result.loc[item,'confidence'] = p_confindenc
            
        aporior_item = item_set(aporior_item_temp)
        
    return result
        

In [8]:
df = DataFrame(np.random.randint(2,size=(10,4)),columns=list('ABCD'))
df.sum(axis=0)

A    4
B    6
C    5
D    7
dtype: int64

In [16]:
%time
support = 0.2
confidence = 0
print('本次Aporior关联算法支持度为【{}】,置信度为【{}】'.format(support,confidence))
aporior_rule_1(df,support,confidence,ms='--')

Wall time: 0 ns
本次Aporior关联算法支持度为【0.2】,置信度为【0】
进行第1次关联，共4个item，分别是['A', 'B', 'C', 'D']
进行第2次关联，共6个item，分别是['AB', 'AC', 'AD', 'BC', 'BD', 'CD']
进行第3次关联，共4个item，分别是['ABC', 'ABD', 'ACD', 'BCD']
进行第4次关联，共1个item，分别是['ABCD']


,support,confidence
A,0.4,1
B,0.6,1
C,0.5,1
D,0.7,1
A--B,0.3,0.5
A--D,0.2,0.285714
B--C,0.3,0.6
B--D,0.5,0.714286
C--D,0.5,0.714286
A--B--D,0.2,0.285714


In [21]:
%time
support = 0.2
confidence = 0.7
print('本次Aporior关联算法支持度为【{}】,置信度为【{}】'.format(support,confidence))
aporior_rule(df,support,confidence,ms='**')

Wall time: 0 ns
本次Aporior关联算法支持度为【0.2】,置信度为【0.7】
进行第1次关联，共4个item，分别是['A', 'B', 'C', 'D']
进行第2次关联，共6个item，分别是['AB', 'AC', 'AD', 'BC', 'BD', 'CD']
进行第3次关联，共4个item，分别是['ABC', 'ABD', 'ACD', 'BCD']
进行第4次关联，共1个item，分别是['ABCD']


,support,confidence
A,0.4,1
B,0.6,1
C,0.5,1
D,0.7,1
B**D,0.5,0.714286
C**D,0.5,0.714286
